In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from vecstack import stacking
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge, Lasso

In [ ]:
train = pd.read_csv("/kaggle/input/tabular-playground-series-jan-2021/train.csv")
test = pd.read_csv("/kaggle/input/tabular-playground-series-jan-2021/test.csv")
sample_submission = pd.read_csv("/kaggle/input/tabular-playground-series-jan-2021/sample_submission.csv")

In [ ]:
train = train.drop('id', axis=1)
test = test.drop('id', axis=1)
X = train.drop('target', axis=1)
y = train.target

## Stacking

In [ ]:
estimators = [
    RandomForestRegressor(n_estimators=500),
    GradientBoostingRegressor(n_estimators=500),
    Ridge(),
    Lasso(),
    LGBMRegressor(bagging_fraction=0.82, bagging_freq=6, cat_smooth=1.0,
              feature_fraction=0.5, lambda_l1=1.075e-05,
              lambda_l2=2.05e-06, learning_rate=0.005,
              max_dept=-1, metric='rmse', min_data_in_leaf=100,
              min_data_per_group=5, min_gain_to_split=0.0,
              min_sum_hessian_in_leaf=0.001, n_estimators=500, num_leaves=246),
    XGBRegressor(colsample_bytree=0.5, alpha=0.01563, learning_rate=0.01, 
                 max_depth=15, min_child_weight=246, n_estimators=500,                                                                  
                 reg_lambda=0.003, subsample=0.7, metric_period=100, silent=1),
    CatBoostRegressor()
]


In [ ]:
S_train, S_test = stacking(estimators,                   
                           X, y, test,   
                           regression=True,
                           metric=mean_squared_error, 
                           n_folds=3,
                           shuffle=True,  
                           random_state=0,
                           verbose=2)

In [ ]:
reg = LinearRegression()
reg = reg.fit(S_train, y)

In [ ]:
y_pred = reg.predict(S_test)
sample_submission['target'] = y_pred
sample_submission.to_csv("stacking_regressor_submission.csv", index=False)